# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [19]:
import requests
import json
import os
import keyring
import sqlite3
import datetime

In [ ]:
# API Configuration
api_key = keyring.get_password("Shopify", "SHOPIFY_API_KEY")
access_token = keyring.get_password("Shopify", "SHOPIFY_API_PASSWORD")
shop_url = keyring.get_password("Shopify", "SHOPIFY_SHOP_URL")
graphql_endpoint = 'https://' + shop_url + '/admin/api/2025-01/graphql.json'

# Headers for API requests
headers = {
    'X-Shopify-Access-Token': access_token,
    'Content-Type': 'application/json'
}

In [8]:
def create_schema(db_path="shopify_data.db"):
    """
    Creates (if not exists) the three tables for storing Shopify customer data.
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Table for basic customer info
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS customers (
            id TEXT PRIMARY KEY,
            email TEXT,
            firstName TEXT,
            lastName TEXT,
            createdAt TEXT,
            displayName TEXT,
            phone TEXT,
            lifetimeDuration TEXT,
            note TEXT,
            numberOfOrders INTEGER,
            updatedAt TEXT,
            verifiedEmail BOOLEAN,
            amountSpent REAL
        )
    ''')
    
    # Table for customer addresses
    # Note that only the default address in Shopify has its own "id".
    # For non-default addresses, we can store a generated row ID
    # and set is_default = 0. 
    # Alternatively, you can store all addresses using rowid automatically
    # and only store the actual Shopify ID for the default address.
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS customer_addresses (
            address_id TEXT PRIMARY KEY,  -- For default addresses, use Shopify's address ID
                                          -- For normal addresses, you might use a synthetic key
            customer_id TEXT,
            address1 TEXT,
            address2 TEXT,
            city TEXT,
            province TEXT,
            country TEXT,
            zip TEXT,
            is_default BOOLEAN,
            company TEXT,
            phone TEXT,
            FOREIGN KEY(customer_id) REFERENCES customers(id)
        )
    ''')
    
    # Table for customer tags
    # Each tag is stored as a separate row linked to the customer
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS customer_tags (
            customer_id TEXT,
            tag TEXT,
            FOREIGN KEY(customer_id) REFERENCES customers(id)
        )
    ''')
    
    conn.commit()
    conn.close()

In [5]:


#Customer fields to retrieve
customer_fields = '''
    id
    email
    firstName
    lastName
    createdAt
    displayName
    phone
    lifetimeDuration
    note
    numberOfOrders
    tags
    updatedAt
    verifiedEmail
    addresses {
        address1
        address2
        city
        province
        country
        zip
    }
    amountSpent {
        amount
    }
    defaultAddress {
        address1
        address2
        city
        company
        id
        phone
        zip
    }
'''




In [16]:
def fetch_all_customers():
    customers = []
    has_next_page = True
    cursor = None
    
    while has_next_page:
        if cursor:
            query = f'''
            {{
              customers(first: 10, after: "{cursor}") {{
                pageInfo {{
                  hasNextPage
                  endCursor
                }}
                edges {{
                  node {{
                    {customer_fields}
                  }}
                }}
              }}
            }}
            '''
        else:
            query = f'''
            {{
              customers(first: 10) {{
                pageInfo {{
                  hasNextPage
                  endCursor
                }}
                edges {{
                  node {{
                    {customer_fields}
                  }}
                }}
              }}
            }}
            '''
            
        try:
            response = requests.post(graphql_endpoint, json={'query': query}, headers=headers)
            response.raise_for_status()
            result = response.json()
            
            page_customers = result['data']['customers']['edges']
            for edge in page_customers:
                customers.append(edge['node'])
            
            has_next_page = result['data']['customers']['pageInfo']['hasNextPage']
            cursor = result['data']['customers']['pageInfo']['endCursor']
            
            #print(f"Fetched {len(page_customers)} customers. Total so far: {len(customers)}")
            
        except Exception as e:
            print(f"Error fetching customers: {e}")
            break
    
    return customers

In [9]:
def store_customers_in_db(customers, db_path="shopify_data.db"):
    """
    Inserts or updates the customer data into the SQLite database.
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    for cust in customers:
        customer_id = cust.get('id', '')
        email = cust.get('email')
        first_name = cust.get('firstName')
        last_name = cust.get('lastName')
        created_at = cust.get('createdAt')
        display_name = cust.get('displayName')
        phone = cust.get('phone')
        lifetime_duration = cust.get('lifetimeDuration')
        note = cust.get('note')
        number_of_orders = cust.get('numberOfOrders', 0)
        tags_list = cust.get('tags', [])
        updated_at = cust.get('updatedAt')
        verified_email = cust.get('verifiedEmail', False)
        
        # amountSpent is a dict with { 'amount': ... }
        amount_spent = cust.get('amountSpent', {}).get('amount', 0.0)
        
        # Insert or replace the customer record
        cursor.execute('''
            INSERT OR REPLACE INTO customers (
                id, email, firstName, lastName, createdAt,
                displayName, phone, lifetimeDuration, note,
                numberOfOrders, updatedAt, verifiedEmail, amountSpent
            )
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            customer_id, email, first_name, last_name, created_at,
            display_name, phone, lifetime_duration, note,
            number_of_orders, updated_at, verified_email, amount_spent
        ))
        
        # Handle tags
        # First, delete any existing tags for this customer to avoid duplicates
        cursor.execute('DELETE FROM customer_tags WHERE customer_id = ?', (customer_id,))
        for tag in tags_list:
            cursor.execute('''
                INSERT INTO customer_tags (customer_id, tag)
                VALUES (?, ?)
            ''', (customer_id, tag))
        
        # Insert addresses
        # First, delete existing addresses for this customer (optional approach)
        cursor.execute('DELETE FROM customer_addresses WHERE customer_id = ?', (customer_id,))
        
        addresses = cust.get('addresses', [])
        default_address = cust.get('defaultAddress')
        
        # Mark whichever address is defaultAddress
        default_addr_id = default_address['id'] if default_address else None
        
        # Insert each address
        for addr in addresses:
            # If there's no unique ID for the non-default addresses, generate a synthetic key
            # For example, we can do: "gid://(customer_id)/(some index)". 
            # Or let SQLite generate a rowid and store that as primary key. 
            # Here, we'll create a simple synthetic ID if it's not default.
            
            is_default = False
            address_id = None
            if default_address and (
                addr.get('address1') == default_address.get('address1') and
                addr.get('address2') == default_address.get('address2') and
                addr.get('city') == default_address.get('city') and
                addr.get('zip') == default_address.get('zip') and
                addr.get('phone') == default_address.get('phone')
            ):
                # This is the default address
                is_default = True
                address_id = default_addr_id
            else:
                # Create a synthetic address_id
                address_id = f"{customer_id}-addr-{addresses.index(addr)}"
            
            cursor.execute('''
                INSERT OR REPLACE INTO customer_addresses (
                    address_id, customer_id, address1, address2, city, province,
                    country, zip, is_default, company, phone
                )
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                address_id,
                customer_id,
                addr.get('address1'),
                addr.get('address2'),
                addr.get('city'),
                addr.get('province'),
                addr.get('country'),
                addr.get('zip'),
                int(is_default),
                # The "company" field is only in defaultAddress in your schema, handle carefully
                default_address.get('company') if is_default and default_address else None,
                addr.get('phone')
            ))
    
    conn.commit()
    conn.close()

In [13]:
create_schema("shopify_data.db")

In [17]:
all_customers = fetch_all_customers()

Fetched 10 customers. Total so far: 10
Fetched 10 customers. Total so far: 20
Fetched 10 customers. Total so far: 30
Fetched 10 customers. Total so far: 40
Fetched 10 customers. Total so far: 50
Fetched 10 customers. Total so far: 60
Fetched 10 customers. Total so far: 70
Fetched 10 customers. Total so far: 80
Fetched 10 customers. Total so far: 90
Fetched 10 customers. Total so far: 100
Fetched 10 customers. Total so far: 110
Fetched 10 customers. Total so far: 120
Fetched 10 customers. Total so far: 130
Fetched 10 customers. Total so far: 140
Fetched 10 customers. Total so far: 150
Fetched 10 customers. Total so far: 160
Fetched 10 customers. Total so far: 170
Fetched 10 customers. Total so far: 180
Fetched 10 customers. Total so far: 190
Fetched 10 customers. Total so far: 200
Fetched 10 customers. Total so far: 210
Fetched 10 customers. Total so far: 220
Fetched 10 customers. Total so far: 230
Fetched 10 customers. Total so far: 240
Fetched 10 customers. Total so far: 250
Fetched 1

In [18]:
store_customers_in_db(all_customers, "shopify_data.db")

In [24]:
def fetch_new_or_updated_customers():
    customers = []
    has_next_page = True
    cursor = None

    # Step 1: Calculate the date 14 days ago using timezone-aware datetime
    fourteen_days_ago = datetime.datetime.now(datetime.timezone.utc) - datetime.timedelta(days=14)
    # Step 2: Format the date in ISO 8601 format
    formatted_date = fourteen_days_ago.isoformat()

    while has_next_page:
        if cursor:
            query = f'''
            {{
              customers(first: 10, after: "{cursor}", query: "createdAt > \\"{formatted_date}\\" OR updatedAt > \\"{formatted_date}\\"") {{
                pageInfo {{
                  hasNextPage
                  endCursor
                }}
                edges {{
                  node {{
                    {customer_fields}
                  }}
                }}
              }}
            }}
            '''
        else:
            query = f'''
            {{
              customers(first: 10, query: "createdAt > \\"{formatted_date}\\" OR updatedAt > \\"{formatted_date}\\"") {{
                pageInfo {{
                  hasNextPage
                  endCursor
                }}
                edges {{
                  node {{
                    {customer_fields}
                  }}
                }}
              }}
            }}
            '''
                
        try:
            response = requests.post(graphql_endpoint, json={'query': query}, headers=headers)
            response.raise_for_status()
            result = response.json()

            # Check for errors in the response
            if 'errors' in result:
                print(f"GraphQL errors: {result['errors']}")
                break

            page_customers = result['data']['customers']['edges']
            for edge in page_customers:
                customers.append(edge['node'])

            has_next_page = result['data']['customers']['pageInfo']['hasNextPage']
            cursor = result['data']['customers']['pageInfo']['endCursor']

            print(f"Fetched {len(page_customers)} customers. Total so far: {len(customers)}")
                
        except Exception as e:
            print(f"Error fetching customers: {e}")
            break
    
    return customers



In [25]:
fetch_new_or_updated_customers()

Fetched 0 customers. Total so far: 0


[]

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
def foo(): pass

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()